In [20]:
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pandas as pd
import matplotlib.pyplot as plt
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pyspark.sql.functions as F
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#change RAM allocation, current = 8GB
#change Core allocation, current = 3 Cores
spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "8g")\
.config("spark.driver.cores", '3')\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 14:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/03 14:54:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("Community Area", StringType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [10]:
#set your local dataset path
dataSetPath = '/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/Data-Preprocessing/Crime-1.csv'

In [11]:
dataset = spark.read.option("header", "True")\
        .option("schema", crimes_schema)\
        .csv(dataSetPath)

### Have a look at the data schema:

In [12]:
dataset.limit(19).toPandas().head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5741943,HN549294,08/25/2007 09:22:18 AM,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,FALSE,FALSE,...,49,1,08A,None,None,2007,08/17/2015 03:03:40 PM,None,None,None
1,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,TRUE,FALSE,...,36,19,01A,1141387,1913179,2021,11/18/2023 03:39:49 PM,41.91783806,-87.75596897,"(41.917838056, -87.755968972)"
2,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,TRUE,FALSE,...,50,13,01A,1152781,1941458,2021,11/18/2023 03:39:49 PM,41.99521944,-87.71335491,"(41.995219444, -87.713354912)"
3,13279676,JG507211,11/09/23 7:30,019XX W BYRON ST,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,FALSE,FALSE,...,47,5,05,1162518,1925906,2023,11/18/2023 03:39:49 PM,41.95234509,-87.67797506,"(41.952345086, -87.677975059)"
4,13274752,JG501049,11/12/23 7:59,086XX S COTTAGE GROVE AVE,0454,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,TRUE,FALSE,...,6,44,08B,1183071,1847869,2023,12/09/23 15:41,41.73775077,-87.60485591,"(41.737750767, -87.604855911)"


### Count the total number of instance before preprocessing:

In [13]:
dataset.count()

999

### Drop the unused columns:


In [14]:
dataset = dataset.drop('ID')\
            .drop('Case Number')\
            .drop('FBI Code')\
            .drop('Updated On')\
            .drop('IUCR')\
            .drop('X Coordinate')\
            .drop('Y Coordinate')\
            .drop('Location')

In [15]:
# Dropping some NA values for now
dataset = dataset.na.drop()
dataset.count()

980

In [27]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

### Add the "DateSimple" column to dataframe:

In [28]:
dataset=dataset\
            .withColumn("DateSimple", F.to_date(F.split(dataset.Date, " ")[0], "MM/dd/yyyy"))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month
0,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,HOMICIDE,FIRST DEGREE MURDER,STREET,TRUE,FALSE,2515,025,36,19,2021,41.91783806,-87.75596897,2021-05-24,2,5
1,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,TRUE,FALSE,1711,017,50,13,2021,41.99521944,-87.71335491,2021-06-26,7,6
2,11/09/23 7:30,019XX W BYRON ST,BURGLARY,UNLAWFUL ENTRY,APARTMENT,FALSE,FALSE,1922,019,47,5,2023,41.95234509,-87.67797506,0023-11-09,5,11
3,11/12/23 7:59,086XX S COTTAGE GROVE AVE,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,TRUE,FALSE,0632,006,6,44,2023,41.73775077,-87.60485591,0023-11-12,1,11
4,09/06/23 17:00,002XX N Wells st,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),FALSE,FALSE,0122,001,42,32,2023,41.88601806,-87.63393788,0023-09-06,4,9


### Add the "Week" column to dataframe:

In [29]:
dataset=dataset.withColumn("Week", F.dayofweek(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month
0,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,HOMICIDE,FIRST DEGREE MURDER,STREET,TRUE,FALSE,2515,025,36,19,2021,41.91783806,-87.75596897,2021-05-24,2,5
1,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,TRUE,FALSE,1711,017,50,13,2021,41.99521944,-87.71335491,2021-06-26,7,6
2,11/09/23 7:30,019XX W BYRON ST,BURGLARY,UNLAWFUL ENTRY,APARTMENT,FALSE,FALSE,1922,019,47,5,2023,41.95234509,-87.67797506,0023-11-09,5,11
3,11/12/23 7:59,086XX S COTTAGE GROVE AVE,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,TRUE,FALSE,0632,006,6,44,2023,41.73775077,-87.60485591,0023-11-12,1,11
4,09/06/23 17:00,002XX N Wells st,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),FALSE,FALSE,0122,001,42,32,2023,41.88601806,-87.63393788,0023-09-06,4,9


### Add the "Month" column to dataframe

In [30]:
dataset=dataset.withColumn("Month", F.month(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month
0,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,HOMICIDE,FIRST DEGREE MURDER,STREET,TRUE,FALSE,2515,025,36,19,2021,41.91783806,-87.75596897,2021-05-24,2,5
1,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,TRUE,FALSE,1711,017,50,13,2021,41.99521944,-87.71335491,2021-06-26,7,6
2,11/09/23 7:30,019XX W BYRON ST,BURGLARY,UNLAWFUL ENTRY,APARTMENT,FALSE,FALSE,1922,019,47,5,2023,41.95234509,-87.67797506,0023-11-09,5,11
3,11/12/23 7:59,086XX S COTTAGE GROVE AVE,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,TRUE,FALSE,0632,006,6,44,2023,41.73775077,-87.60485591,0023-11-12,1,11
4,09/06/23 17:00,002XX N Wells st,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),FALSE,FALSE,0122,001,42,32,2023,41.88601806,-87.63393788,0023-09-06,4,9


In [32]:
### Add the "Day" column to dataframe

In [33]:
#1- SUNDAY , 2 - MONDAY .... 7- SATURDAY
dataset=dataset.withColumn("Day", F.dayofweek(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month,Day
0,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,HOMICIDE,FIRST DEGREE MURDER,STREET,TRUE,FALSE,2515,025,36,19,2021,41.91783806,-87.75596897,2021-05-24,2,5,2
1,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,TRUE,FALSE,1711,017,50,13,2021,41.99521944,-87.71335491,2021-06-26,7,6,7
2,11/09/23 7:30,019XX W BYRON ST,BURGLARY,UNLAWFUL ENTRY,APARTMENT,FALSE,FALSE,1922,019,47,5,2023,41.95234509,-87.67797506,0023-11-09,5,11,5
3,11/12/23 7:59,086XX S COTTAGE GROVE AVE,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,TRUE,FALSE,0632,006,6,44,2023,41.73775077,-87.60485591,0023-11-12,1,11,1
4,09/06/23 17:00,002XX N Wells st,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),FALSE,FALSE,0122,001,42,32,2023,41.88601806,-87.63393788,0023-09-06,4,9,4
